#  Personal Capstone- AZ

Use the Notebook to build the code to scrape the following Wikipedia page, https://www.unitedstateszipcodes.org/az/, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe 

To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
 * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.



In [1]:
!pip install geopy

# import packages
import numpy as np # library to handle data in a vectorized manner
import csv
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


     |████████████████████████████████| 112kB 6.4MB/s eta 0:00:01
Libraries imported.


### 1) Create a dataframe

In [2]:
df= pd.read_csv("PH_ZIP.csv", ) 

In [3]:

print(df.shape)

(42632, 15)


In [4]:
#  create new dataframe to focus on stats that contain "AZ"

df_AZ = df[df['state'].str.contains("AZ")].reset_index(drop=True)
df_AZ.head()


,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,85001,PO BOX,0,Phoenix,NaN,NaN,AZ,Maricopa County,America/Phoenix,602,NaN,US,33.45,-112.06,759
1,85002,PO BOX,0,Phoenix,NaN,NaN,AZ,Maricopa County,America/Phoenix,602,NaN,US,33.45,-112.06,345
2,85003,STANDARD,0,Phoenix,NaN,NaN,AZ,Maricopa County,America/Phoenix,"480,623,602,928",NaN,US,33.45,-112.08,6230
3,85004,STANDARD,0,Phoenix,NaN,NaN,AZ,Maricopa County,America/Phoenix,"520,928,480,602,623",NaN,US,33.44,-112.07,3930
4,85005,PO BOX,0,Phoenix,NaN,NaN,AZ,Maricopa County,America/Phoenix,602,NaN,US,33.44,-112.12,1050


In [5]:
GREATER_PHX =['Phoenix','Glendale','Tempe','Scottsdale','Paradise Valley']

In [6]:
#  create new dataframe contain only "Phoenix"

maricopa = df_AZ[df_AZ['county'].isin(['Maricopa County'])].reset_index(drop=True)
maricopa.head()

                 


,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,85001,PO BOX,0,Phoenix,NaN,NaN,AZ,Maricopa County,America/Phoenix,602,NaN,US,33.45,-112.06,759
1,85002,PO BOX,0,Phoenix,NaN,NaN,AZ,Maricopa County,America/Phoenix,602,NaN,US,33.45,-112.06,345
2,85003,STANDARD,0,Phoenix,NaN,NaN,AZ,Maricopa County,America/Phoenix,"480,623,602,928",NaN,US,33.45,-112.08,6230
3,85004,STANDARD,0,Phoenix,NaN,NaN,AZ,Maricopa County,America/Phoenix,"520,928,480,602,623",NaN,US,33.44,-112.07,3930
4,85005,PO BOX,0,Phoenix,NaN,NaN,AZ,Maricopa County,America/Phoenix,602,NaN,US,33.44,-112.12,1050


In [7]:
#  drop unnecessary columns

maricopa.drop(df.columns[[1,2,4,5,8,9,10,11,14]], axis=1 ,inplace = True)
maricopa.tail()

,zip,primary_city,state,county,latitude,longitude
203,85388,Surprise,AZ,Maricopa County,33.60,-112.43
204,85390,Wickenburg,AZ,Maricopa County,33.90,-112.90
205,85392,Avondale,AZ,Maricopa County,33.47,-112.29
206,85395,Goodyear,AZ,Maricopa County,33.47,-112.39
207,85396,Buckeye,AZ,Maricopa County,33.53,-112.70


In [8]:
maricopa.shape

(208, 6)

In [9]:
phoenix_data= maricopa[['primary_city','latitude','longitude']]

phoenix_data.head()



,primary_city,latitude,longitude
0,Phoenix,33.45,-112.06
1,Phoenix,33.45,-112.06
2,Phoenix,33.45,-112.08
3,Phoenix,33.44,-112.07
4,Phoenix,33.44,-112.12


In [10]:
phoenix_data.shape

(208, 3)

In [30]:
address = 'Phoenix, AZ'

geolocator = Nominatim(user_agent="ph_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Phoenix, AZ are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Phoenix, AZ are 33.4484367, -112.0741417.


In [31]:
# create map of Manhattan using latitude and longitude values
map_maricopa = folium.Map(location=[latitude, longitude], zoom_start=11)

#add markers to map
for latitude, longitude, label in zip(maricopa['latitude'], maricopa['longitude'], maricopa['zip']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_maricopa)  

map_maricopa

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'BSKHYD0O00O3RIMY4DAIDFTRTZDQYO35MYWFFL2QVA14TFBY' # your Foursquare ID
CLIENT_SECRET = 'L0JKAQQLQ4B42WZJSFUEM0N02Z2NQZLRMG1VRCOUM2VE0YGY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BSKHYD0O00O3RIMY4DAIDFTRTZDQYO35MYWFFL2QVA14TFBY
CLIENT_SECRET:L0JKAQQLQ4B42WZJSFUEM0N02Z2NQZLRMG1VRCOUM2VE0YGY


A quick Google search tells me that Paradise Valley is one of the pricest neighborhoods in the Greater Phoenix area. Let's find its coordinates

In [14]:
maricopa[maricopa['primary_city'].str.contains('Paradise') ]

,zip,primary_city,state,county,latitude,longitude
118,85253,Paradise Valley,AZ,Maricopa County,33.54,-111.96


In [34]:
maricopa[maricopa['zip']==85253]

,zip,primary_city,state,county,latitude,longitude
118,85253,Paradise Valley,AZ,Maricopa County,33.54,-111.96


In [35]:
#let's assign the coordinates. We know Paradise Valley is located in row 84.

PV_lat = maricopa.loc[118,'latitude']
PV_long = maricopa.loc[118,'longitude']

print(PV_lat)
print(PV_long)



33.54
-111.96


#### Now, let's get the top 100 venues that are in Maricopa County within a radius of 500 meters.


In [36]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID = 'BSKHYD0O00O3RIMY4DAIDFTRTZDQYO35MYWFFL2QVA14TFBY' # your Foursquare ID
CLIENT_SECRET = 'L0JKAQQLQ4B42WZJSFUEM0N02Z2NQZLRMG1VRCOUM2VE0YGY' # your Foursquare Secret
ACCESS_TOKEN = 'UWR1W1O2W0DWXUI1WWULR3PRX5ST0NVEOASHYC2Z2GRFWES5' # your FourSquare Access Token
VERSION = '20180604'



 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    PV_lat, 
    PV_long, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=BSKHYD0O00O3RIMY4DAIDFTRTZDQYO35MYWFFL2QVA14TFBY&client_secret=L0JKAQQLQ4B42WZJSFUEM0N02Z2NQZLRMG1VRCOUM2VE0YGY&v=20180604&ll=33.54,-111.96&radius=500&limit=100'

In [37]:
results = requests.get(url).json()

In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [39]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)# flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Mummy Mountain,Mountain,33.543268,-111.961518
1,Tyner's Trail,Trail,33.538176,-111.962928


In [29]:
nearby_venues.shape

(2, 4)

## 2. Explore Neighborhoods in Manhattan


In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _manhattan_venues_.


In [41]:
# type your answer here
phoenix_venues = getNearbyVenues(names= maricopa['primary_city'],
                                  latitudes=maricopa['latitude'],
                                   longitudes=maricopa['longitude']
                                  )

Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
Phoenix
New River
Phoenix
Phoenix
Phoenix
Phoenix
Chandler Heights
Queen Creek
Tortilla Flat
Mesa
Mesa
Mesa
Mesa
Mesa
Mesa
Mesa
Mesa
Mesa
Mesa
Mesa
Mesa
Mesa
Mesa
Mesa
Mesa
Chandler
Chandler
Chandler
Chandler Heights
Gilbert
Gilbert
Higley
Queen Creek
Queen Creek
Queen Creek
Chandler
Chandler
Chandler
Chandler
Scottsdale
Scottsdale
Scottsdale
Paradise Valley
Scottsdale
Scottsdale
Scottsdal

#### Let's check the size of the resulting dataframe


In [42]:
print(phoenix_venues.shape)
phoenix_venues.head()

(1216, 7)


,City,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Phoenix,33.45,-112.06,Children's Museum of Phoenix,33.450438,-112.064609,Museum
1,Phoenix,33.45,-112.06,Mrs. White's Golden Rule,33.447338,-112.063381,Southern / Soul Food Restaurant
2,Phoenix,33.45,-112.06,SpringHill Suites Phoenix Downtown,33.451715,-112.062892,Hotel
3,Phoenix,33.45,-112.06,Redbox,33.451759,-112.059502,Video Store
4,Phoenix,33.45,-112.06,Dollar General,33.451017,-112.055384,Discount Store


In [43]:
phoenix_venues.groupby('City').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Apache Junction,3,3,3,3,3,3
Avondale,5,5,5,5,5,5
Cashion,5,5,5,5,5,5
Chandler,48,48,48,48,48,48
Chandler Heights,3,3,3,3,3,3
El Mirage,1,1,1,1,1,1
Fountain Hills,7,7,7,7,7,7
Gilbert,47,47,47,47,47,47
Glendale,58,58,58,58,58,58


#### Let's find out how many unique categories can be curated from all the returned venues


In [44]:
print('There are {} uniques categories.'.format(len(phoenix_venues['Venue Category'].unique())))

There are 218 uniques categories.


## 3. Analyze Each Neighborhood


In [45]:
# one hot encoding
phoenix_onehot = pd.get_dummies(phoenix_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
phoenix_onehot['City'] = phoenix_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [phoenix_onehot.columns[-1]] + list(phoenix_onehot.columns[:-1])
phoenix_onehot = phoenix_onehot[fixed_columns]

phoenix_onehot.head()

,City,ATM,Accessories Store,Advertising Agency,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Phoenix,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Phoenix,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Phoenix,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Phoenix,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,Phoenix,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.


In [46]:
phoenix_onehot.shape

(1216, 219)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [91]:
phoenix_grouped = phoenix_onehot.groupby('City').mean().reset_index()
phoenix_grouped

,City,ATM,Accessories Store,Advertising Agency,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Apache Junction,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Avondale,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Cashion,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Chandler,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833
4,Chandler Heights,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,El Mirage,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Fountain Hills,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Gilbert,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.021277,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000
8,Glendale,0.000000,0.000000,0.000000,0.000000,0.051724,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000
9,Goodyear,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


phoenix_grouped.shape

#### Let's print each neighborhood along with the top 5 most common venues


In [97]:
num_top_venues = 5

for hood in phoenix_grouped['City']:
    print("----"+hood+"----")
    temp = phoenix_grouped[phoenix_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Apache Junction----
           venue  freq
0      BBQ Joint  0.33
1   Intersection  0.33
2           Park  0.33
3    Opera House  0.00
4  Moving Target  0.00


----Avondale----
            venue  freq
0    Home Service   0.4
1            Park   0.4
2  Breakfast Spot   0.2
3             ATM   0.0
4          Notary   0.0


----Cashion----
       venue  freq
0       Park   0.4
1  Pet Store   0.2
2  Multiplex   0.2
3     Notary   0.2
4        ATM   0.0


----Chandler----
                  venue  freq
0        Ice Cream Shop  0.06
1                  Park  0.06
2             Pet Store  0.04
3  Gym / Fitness Center  0.04
4    Mexican Restaurant  0.04


----Chandler Heights----
                    venue  freq
0      Mexican Restaurant  0.67
1  Furniture / Home Store  0.33
2                     ATM  0.00
3       Mobile Phone Shop  0.00
4           Movie Theater  0.00


----El Mirage----
           venue  freq
0           Farm   1.0
1            ATM   0.0
2  Movie Theater   0.0
3  Moving Tar

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [142]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['City'] = phoenix_grouped['City']

for ind in np.arange(phoenix_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(phoenix_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Apache Junction,BBQ Joint,Park,Intersection,Yoga Studio,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market
1,Avondale,Home Service,Park,Breakfast Spot,Yoga Studio,Fabric Shop,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market
2,Cashion,Park,Pet Store,Multiplex,Notary,Fabric Shop,Food,Fondue Restaurant,Flower Shop,Flea Market,Fish & Chips Shop
3,Chandler,Park,Ice Cream Shop,Coffee Shop,Intersection,Mexican Restaurant,Construction & Landscaping,Gym / Fitness Center,Pet Store,Pizza Place,Pharmacy
4,Chandler Heights,Mexican Restaurant,Furniture / Home Store,Yoga Studio,Food Truck,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [143]:
# set number of clusters
kclusters = 6

phoenix_grouped_clustering = phoenix_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(phoenix_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 1, 4, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [144]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

phoenix_merged = phoenix_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
phoenix_merged = phoenix_merged.join(neighborhoods_venues_sorted.set_index('City'), on='primary_city')

phoenix_merged.head()

,primary_city,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Phoenix,33.45,-112.06,1.0,American Restaurant,Mexican Restaurant,Pizza Place,Convenience Store,Coffee Shop,Hotel,Park,Rental Car Location,Intersection,Sandwich Place
1,Phoenix,33.45,-112.06,1.0,American Restaurant,Mexican Restaurant,Pizza Place,Convenience Store,Coffee Shop,Hotel,Park,Rental Car Location,Intersection,Sandwich Place
2,Phoenix,33.45,-112.08,1.0,American Restaurant,Mexican Restaurant,Pizza Place,Convenience Store,Coffee Shop,Hotel,Park,Rental Car Location,Intersection,Sandwich Place
3,Phoenix,33.44,-112.07,1.0,American Restaurant,Mexican Restaurant,Pizza Place,Convenience Store,Coffee Shop,Hotel,Park,Rental Car Location,Intersection,Sandwich Place
4,Phoenix,33.44,-112.12,1.0,American Restaurant,Mexican Restaurant,Pizza Place,Convenience Store,Coffee Shop,Hotel,Park,Rental Car Location,Intersection,Sandwich Place


In [145]:
phoenix_merged.tail()# check the last columns! 

,primary_city,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
203,Surprise,33.60,-112.43,0.0,Pizza Place,Intersection,Spa,Garden Center,Gun Shop,Shopping Mall,Park,Home Service,Flower Shop,Flea Market
204,Wickenburg,33.90,-112.90,0.0,Mountain,Park,Business Service,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market
205,Avondale,33.47,-112.29,0.0,Home Service,Park,Breakfast Spot,Yoga Studio,Fabric Shop,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market
206,Goodyear,33.47,-112.39,1.0,Department Store,Pet Store,Clothing Store,Sushi Restaurant,Street Food Gathering,Fried Chicken Joint,Massage Studio,Shoe Store,Electronics Store,Nail Salon
207,Buckeye,33.53,-112.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
phoenix_merged.shape

(208, 14)

Let's clean the data.  Looking at the tail end of the dataframe, it looks like there are null values, meaning there aren't locations near those zipcodes. Let's remove them before continuing.

In [147]:
## Looking at the tail

phoenix_merged2 = phoenix_merged
phoenix_merged2.dropna(inplace = True )
phoenix_merged2.tail()



,primary_city,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
202,Surprise,33.71,-112.40,0.0,Pizza Place,Intersection,Spa,Garden Center,Gun Shop,Shopping Mall,Park,Home Service,Flower Shop,Flea Market
203,Surprise,33.60,-112.43,0.0,Pizza Place,Intersection,Spa,Garden Center,Gun Shop,Shopping Mall,Park,Home Service,Flower Shop,Flea Market
204,Wickenburg,33.90,-112.90,0.0,Mountain,Park,Business Service,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market
205,Avondale,33.47,-112.29,0.0,Home Service,Park,Breakfast Spot,Yoga Studio,Fabric Shop,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market
206,Goodyear,33.47,-112.39,1.0,Department Store,Pet Store,Clothing Store,Sushi Restaurant,Street Food Gathering,Fried Chicken Joint,Massage Studio,Shoe Store,Electronics Store,Nail Salon


In [148]:
phoenix_merged2.shape

(192, 14)

In [149]:
phoenix_merged2['Cluster Labels'] =phoenix_merged2['Cluster Labels'].astype(int)

Finally, let's visualize the resulting clusters


In [150]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(phoenix_merged2['latitude'], phoenix_merged['longitude'], phoenix_merged2['primary_city'], phoenix_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


Cluster 1

In [157]:
phoenix_merged2.loc[phoenix_merged2['Cluster Labels'] == 0, phoenix_merged2.columns[[0] + list(range(4, phoenix_merged2.shape[1]))]]

,primary_city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
147,Apache Junction,BBQ Joint,Park,Intersection,Yoga Studio,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market
169,Avondale,Home Service,Park,Breakfast Spot,Yoga Studio,Fabric Shop,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market
172,Cashion,Park,Pet Store,Multiplex,Notary,Fabric Shop,Food,Fondue Restaurant,Flower Shop,Flea Market,Fish & Chips Shop
186,Wickenburg,Mountain,Park,Business Service,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market
191,Surprise,Pizza Place,Intersection,Spa,Garden Center,Gun Shop,Shopping Mall,Park,Home Service,Flower Shop,Flea Market
195,Surprise,Pizza Place,Intersection,Spa,Garden Center,Gun Shop,Shopping Mall,Park,Home Service,Flower Shop,Flea Market
196,Surprise,Pizza Place,Intersection,Spa,Garden Center,Gun Shop,Shopping Mall,Park,Home Service,Flower Shop,Flea Market
202,Surprise,Pizza Place,Intersection,Spa,Garden Center,Gun Shop,Shopping Mall,Park,Home Service,Flower Shop,Flea Market
203,Surprise,Pizza Place,Intersection,Spa,Garden Center,Gun Shop,Shopping Mall,Park,Home Service,Flower Shop,Flea Market
204,Wickenburg,Mountain,Park,Business Service,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market


Cluster 2

In [156]:
phoenix_merged2.loc[phoenix_merged2['Cluster Labels'] == 1, phoenix_merged2.columns[[0] + list(range(4, phoenix_merged2.shape[1]))]]

#manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,primary_city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Phoenix,American Restaurant,Mexican Restaurant,Pizza Place,Convenience Store,Coffee Shop,Hotel,Park,Rental Car Location,Intersection,Sandwich Place
1,Phoenix,American Restaurant,Mexican Restaurant,Pizza Place,Convenience Store,Coffee Shop,Hotel,Park,Rental Car Location,Intersection,Sandwich Place
2,Phoenix,American Restaurant,Mexican Restaurant,Pizza Place,Convenience Store,Coffee Shop,Hotel,Park,Rental Car Location,Intersection,Sandwich Place
3,Phoenix,American Restaurant,Mexican Restaurant,Pizza Place,Convenience Store,Coffee Shop,Hotel,Park,Rental Car Location,Intersection,Sandwich Place
4,Phoenix,American Restaurant,Mexican Restaurant,Pizza Place,Convenience Store,Coffee Shop,Hotel,Park,Rental Car Location,Intersection,Sandwich Place
...,...,...,...,...,...,...,...,...,...,...,...
198,Peoria,Fast Food Restaurant,Coffee Shop,Mexican Restaurant,Video Store,Pizza Place,Food Truck,Sporting Goods Shop,ATM,Convenience Store,Grocery Store
199,Peoria,Fast Food Restaurant,Coffee Shop,Mexican Restaurant,Video Store,Pizza Place,Food Truck,Sporting Goods Shop,ATM,Convenience Store,Grocery Store
200,Peoria,Fast Food Restaurant,Coffee Shop,Mexican Restaurant,Video Store,Pizza Place,Food Truck,Sporting Goods Shop,ATM,Convenience Store,Grocery Store
201,Peoria,Fast Food Restaurant,Coffee Shop,Mexican Restaurant,Video Store,Pizza Place,Food Truck,Sporting Goods Shop,ATM,Convenience Store,Grocery Store


Cluster 3

In [161]:
phoenix_merged2.loc[phoenix_merged2['Cluster Labels'] == 2, phoenix_merged2.columns[[0] + list(range(4, phoenix_merged2.shape[1]))]]


,primary_city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
185,Waddell,Intersection,Yoga Studio,Fabric Shop,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market,Fish & Chips Shop
188,Youngtown,Intersection,Yoga Studio,Fabric Shop,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market,Fish & Chips Shop


Cluster 4

In [160]:
phoenix_merged2.loc[phoenix_merged2['Cluster Labels'] == 3, phoenix_merged2.columns[[0] + list(range(4, phoenix_merged2.shape[1]))]]


,primary_city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Queen Creek,Butcher,Yoga Studio,Falafel Restaurant,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market,Fish & Chips Shop
108,Queen Creek,Butcher,Yoga Studio,Falafel Restaurant,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market,Fish & Chips Shop
109,Queen Creek,Butcher,Yoga Studio,Falafel Restaurant,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market,Fish & Chips Shop
110,Queen Creek,Butcher,Yoga Studio,Falafel Restaurant,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market,Fish & Chips Shop


Cluster 5

In [159]:
phoenix_merged2.loc[phoenix_merged2['Cluster Labels'] == 4, phoenix_merged2.columns[[0] + list(range(4, phoenix_merged2.shape[1]))]]


,primary_city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
174,El Mirage,Farm,Yoga Studio,Falafel Restaurant,Food Service,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market


Cluster 6

In [162]:
phoenix_merged2.loc[phoenix_merged2['Cluster Labels'] == 5, phoenix_merged2.columns[[0] + list(range(4, phoenix_merged2.shape[1]))]]


,primary_city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
177,Laveen,Dessert Shop,Yoga Studio,Falafel Restaurant,Food Service,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market
